In [133]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


In [134]:
np.random.seed(100)

In [136]:
# Trying to see how sensitive computed market shares are to the initialized parameters
# All the parameters of interest 
beta = np.array([0.5, -1.7, -0.3])
mu = 0.5
omega = 1

# alpha = -np.exp(mu + omega**2/2)

# The number of firms in the market
J = 3

# The number of product characteristics 
K = 2

# The number of consumers in the market 
N = 2

# Schockes per consumer 
v_p = np.random.normal(0, 1, size = N)

X1 = np.random.uniform(6, 24, size=J)
X2 = np.random.normal(3, 1, size=J)
all_X = np.column_stack((X1, X2))
X0 = np.ones(J)
X = np.column_stack((X0, all_X))

p = np.reshape((np.ones(J)), (1, J))
e = np.random.gumbel(size=N*J)




In [137]:
# Getting the summpation function over the 
# Making a killer function for calculating market shares 
def share(N_cons, J_prod, X_char, price, coef_car, mu, omega, e): 
    X_for_utility = np.repeat(X_char, N_cons, axis=0)
    u = X_for_utility@coef_car + np.ravel((np.reshape((-(np.exp(mu + omega*v_p))), (N_cons, 1)))@price) + e
    u = np.reshape(u, (J_prod, N_cons))
    sum_u = sum(u)
    prob = np.exp(u)/(1 + np.exp(sum_u))
    shares = np.sum(prob, axis=1)
    return(shares)


In [138]:
shares_my_market = share(N, J, X, p, beta, mu, omega, e)
print(shares_my_market)

[6.23464104e-09 1.24739600e-09 3.12083388e-17]


In [154]:
# Now gettting the correct Jacobian amtric with random coefficients and the stupid fucking error term 
# The random coff just to check how to rewrite the jacobian function 

J_ii = np.sum((-(np.exp(mu + omega*v_p)))* shares_my_market[0] - (-(np.exp(mu + omega*v_p)))*shares_my_market[0]**2)  
print(J_ii)

J_ij = np.sum(-(-(np.exp(mu + omega*v_p)))*shares_my_market[0]*shares_my_market[1])
print(J_ij)


-2.8347038766813165e-08
3.5359983016187485e-17


In [157]:

def derivative(J_prod, mu, omega, shares):
    J = np.zeros((J_prod, J_prod))
    for i in range(J.shape[0]):
        for j in range(J.shape[1]):
            if i == j:
                # this is the own product elasticity
                J[i, j] = np.sum((-(np.exp(mu + omega*v_p)))* shares[i] - (-(np.exp(mu + omega*v_p)))*shares[i]**2)
            elif i!= j: 
                # this is the cross product elasticity
                J[i, j] = np.sum(-(-(np.exp(mu + omega*v_p)))*shares[i]*shares[j])
    return J




In [158]:
J = derivative(J, mu, omega, shares_my_market)
print(J)

[[-2.83470388e-08  3.53599830e-17  8.84663996e-25]
 [ 3.53599830e-17 -5.67153470e-09  1.76999177e-25]
 [ 8.84663996e-25  1.76999177e-25 -1.41894937e-16]]


0.1670654382112677
